## Задание 2 - Токенизация | Лемматизация

In [ ]:
!pip install nltk

In [ ]:
!pip install pymorphy2

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet') 
nltk.download('punkt')

In [ ]:
import os
from bs4 import BeautifulSoup

In [39]:
# Контент включается в себя заголовок и основной текст новостной статьи
content = []

for file_ in os.listdir('links'):
  if (file_.__contains__('.html')):
    path_ = os.path.join('links', file_)
    file_text = open(path_).read()
  
    soup = BeautifulSoup(file_text)
  
    for text in soup.find('h1', {'class': 'tm-article-snippet__title tm-article-snippet__title_h1'}):
      title = text.get_text()
      content.append(title)

    data = soup.find('div', {'xmlns': 'http://www.w3.org/1999/xhtml'})
    for p in data.find_all('p'):
       text = p.get_text()
       content.append(text)

In [160]:
import string
from nltk.corpus import stopwords
import re
 
# В тексте большое количество английских слов, связанных с IT и соответственно
# существенно значимых для поиска, поэтому токены содержат русские и английские слова
def tokenize(content):
    tokens = set() 
    for c in content:
      tokens_ = nltk.word_tokenize(c)
      tokens_ = [i.lower() for i in tokens_]
      tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
      tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

      stop_words = stopwords.words('russian')
      stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
      tokens_ = [i for i in tokens_ if ( i not in stop_words )]

      stop_words2 = stopwords.words('english')
      stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
      tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

      tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

      for t_ in tokens_:
        tokens.add(t_)
 
    return tokens

In [117]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def lemmatize(tokens):
  lemmas = {}
  for token in tokens:
    lemma = morph.parse(token)[0].normal_form
    flag = lemmas.get(lemma)
    if flag is None:
      lemmas[lemma] = token
    else:
      if token not in flag:
        lemmas[lemma] = flag + ' ' + token
  return lemmas

In [161]:
tokens = tokenize(content)

In [154]:
lemmas = lemmatize(tokens)

In [163]:
file_1 = open('tokens.txt', "w")
for token in tokens:
  file_1.write(token + '\n')
file_1.close()

file_2 = open('lemmas.txt', "w")
for key in lemmas:
  res = key + ':'
  for i in lemmas[key]:
    res = res  + i
  file_2.write(res + '\n')
file_2.close()

In [164]:
from google.colab import files
files.download("lemmas.txt")
files.download("tokens.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>